In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [3]:
num_iter = 5
xmax = 2.
zmax = 1.
vmax = 1.

for j = 1:num_iter
    # half in contact, half not
    if j%2==0 
        q0 = vcat([1., 0., 0., 0.], (rand(2) .- .5) .* 2. .* xmax, 0.)
    else
        q0 = vcat([1., 0., 0., 0.], (rand(2) .- .5) .* 2. .* xmax, rand(1) .* zmax)
    end
#     q0 = [1.,0.,0.,0.,0.,0.,0.]
#     v0 = vcat(zeros(3),(rand(3) .- .5) .* 2. .* vmax)
    v0 = vcat(zeros(3),(rand(2) .- .5) .* 2. .* vmax,0.)
    u0 = zeros(sim_data.num_v)
    ctrl! = (u,t,x) -> u[:] = 0.

    set_configuration!(x0, q0)
    set_velocity!(x0, v0)
    setdirty!(x0)
    
    traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
    qnext = traj[1:sim_data.num_q,end]
    vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,end]
    x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,end]
   
    τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);
    τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

    res = hcat(x_sol_exp,x_sol_ip,x_sol_auglag,x_sol_exp.-x_sol_auglag,x_sol_ip.-x_sol_auglag)

    display(res)
end

elapsed time: 21.334203771 seconds
elapsed time: 0.004745108 seconds

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

elapsed time: 0.004443527 seconds
elapsed time: 0.00464523 seconds
elapsed time: 0.004549206 seconds
elapsed time: 0.005385664 seconds
elapsed time: 0.00459753 seconds
elapsed time: 0.0045137 seconds
elapsed time: 0.00420813 seconds
elapsed time: 0.004407088 seconds
elapsed time: 0.004173384 seconds
elapsed time: 0.004297128 seconds
elapsed time: 0.003976501 seconds
elapsed time: 0.003938458 seconds
elapsed time: 0.003959028 seconds
elapsed time: 0.004028399 seconds
elapsed time: 0.004679465 seconds
elapsed time: 0.004506757 seconds
elap

elapsed time: 0.003909674 seconds
elapsed time: 0.004830671 seconds
elapsed time: 0.004373234 seconds
elapsed time: 0.005287366 seconds
elapsed time: 0.004543646 seconds
elapsed time: 0.004784714 seconds
elapsed time: 0.004190276 seconds
elapsed time: 0.004263746 seconds
elapsed time: 0.004186489 seconds
elapsed time: 0.004351352 seconds
elapsed time: 0.010956177 seconds
elapsed time: 0.004166031 seconds
elapsed time: 0.003983093 seconds
elapsed time: 0.004133159 seconds
elapsed time: 0.004415624 seconds
elapsed time: 0.004721684 seconds
elapsed time: 0.00535193 seconds
elapsed time: 0.004665648 seconds
elapsed time: 0.004800773 seconds
elapsed time: 0.004552103 seconds
elapsed time: 0.004648876 seconds
elapsed time: 0.004316094 seconds
elapsed time: 0.004423289 seconds
elapsed time: 0.004375858 seconds
elapsed time: 0.004379675 seconds
elapsed time: 0.00424475 seconds
elapsed time: 0.004139921 seconds
elapsed time: 0.004178865 seconds
elapsed time: 0.004668202 seconds
elapsed time: 0.

elapsed time: 0.003968947 seconds
elapsed time: 0.004669315 seconds
elapsed time: 0.004385346 seconds
elapsed time: 0.005335888 seconds
elapsed time: 0.004300295 seconds
elapsed time: 0.004521564 seconds
elapsed time: 0.004422136 seconds
elapsed time: 0.011182045 seconds
elapsed time: 0.004164688 seconds
elapsed time: 0.004166742 seconds
elapsed time: 0.003948478 seconds
elapsed time: 0.004374997 seconds
elapsed time: 0.003919202 seconds
elapsed time: 0.003913742 seconds
elapsed time: 0.004668573 seconds
elapsed time: 0.004474505 seconds
elapsed time: 0.005356247 seconds
elapsed time: 0.004314561 seconds
elapsed time: 0.00460865 seconds
elapsed time: 0.004284305 seconds
elapsed time: 0.004880106 seconds
elapsed time: 0.004420493 seconds
elapsed time: 0.004353837 seconds
elapsed time: 0.004635811 seconds
elapsed time: 0.004406276 seconds
elapsed time: 0.006843468 seconds
elapsed time: 0.00497307 seconds
elapsed time: 0.007226083 seconds
elapsed time: 0.014231468 seconds
elapsed time: 0.

elapsed time: 0.004338377 seconds
elapsed time: 0.004664095 seconds
elapsed time: 0.004586197 seconds
elapsed time: 0.013279152 seconds
elapsed time: 0.004489634 seconds
elapsed time: 0.006343249 seconds
elapsed time: 0.005217184 seconds
elapsed time: 0.006770559 seconds
elapsed time: 0.006525014 seconds
elapsed time: 0.005120711 seconds
elapsed time: 0.007370827 seconds
elapsed time: 0.004912526 seconds
elapsed time: 0.006993742 seconds
elapsed time: 0.005954652 seconds
elapsed time: 0.005393608 seconds
elapsed time: 0.005248282 seconds
elapsed time: 0.005113698 seconds
elapsed time: 0.005119338 seconds
elapsed time: 0.005134738 seconds
elapsed time: 0.005237041 seconds
elapsed time: 0.005078351 seconds
elapsed time: 0.005328635 seconds
elapsed time: 0.005240869 seconds
elapsed time: 0.00492026 seconds
elapsed time: 0.012990074 seconds
elapsed time: 0.005367639 seconds
elapsed time: 0.004638085 seconds
elapsed time: 0.004358445 seconds
elapsed time: 0.004248327 seconds
elapsed time: 0

elapsed time: 0.004152014 seconds
elapsed time: 0.004642634 seconds
elapsed time: 0.004905002 seconds
elapsed time: 0.005202766 seconds
elapsed time: 0.004609892 seconds
elapsed time: 0.004431974 seconds
elapsed time: 0.004269025 seconds
elapsed time: 0.011682074 seconds
elapsed time: 0.004483833 seconds
elapsed time: 0.004448535 seconds
elapsed time: 0.004388492 seconds
elapsed time: 0.00431868 seconds
elapsed time: 0.004273584 seconds
elapsed time: 0.004279385 seconds
elapsed time: 0.004493882 seconds
elapsed time: 0.005318485 seconds
elapsed time: 0.004859565 seconds
elapsed time: 0.004411025 seconds
elapsed time: 0.003896008 seconds
elapsed time: 0.004357162 seconds
elapsed time: 0.004174697 seconds
elapsed time: 0.004235272 seconds
elapsed time: 0.004411936 seconds
elapsed time: 0.004015234 seconds
elapsed time: 0.00403925 seconds
elapsed time: 0.004265599 seconds
elapsed time: 0.007718307 seconds
elapsed time: 0.006027901 seconds
elapsed time: 0.014191112 seconds
elapsed time: 0.

...

elapsed time: 0.003926896 seconds
elapsed time: 0.004507097 seconds
elapsed time: 0.004412888 seconds
elapsed time: 0.004339769 seconds
elapsed time: 0.00475729 seconds
elapsed time: 0.004176029 seconds
elapsed time: 0.004117428 seconds
elapsed time: 0.003898112 seconds
elapsed time: 0.004972649 seconds
elapsed time: 0.004377672 seconds
elapsed time: 0.004940328 seconds
elapsed time: 0.004276189 seconds
elapsed time: 0.004265408 seconds
elapsed time: 0.00399203 seconds
elapsed time: 0.003940603 seconds
elapsed time: 0.004596957 seconds
elapsed time: 0.004380066 seconds
elapsed time: 0.004369976 seconds
elapsed time: 0.004311986 seconds
elapsed time: 0.004220173 seconds
elapsed time: 0.004511154 seconds
elapsed time: 0.00430306 seconds
elapsed time: 0.004259207 seconds
elapsed time: 0.003963245 seconds
elapsed time: 0.00390722 seconds
elapsed time: 0.003974046 seconds
elapsed time: 0.004194915 seconds
elapsed time: 0.004087161 seconds
elapsed time: 0.004172082 seconds
elapsed time: 0.00

elapsed time: 0.004346613 seconds
elapsed time: 0.004574434 seconds
elapsed time: 0.004576358 seconds
elapsed time: 0.004690604 seconds
elapsed time: 0.004420613 seconds
elapsed time: 0.004665247 seconds
elapsed time: 0.00418211 seconds
elapsed time: 0.004344148 seconds
elapsed time: 0.004344078 seconds
elapsed time: 0.00425531 seconds
elapsed time: 0.004119502 seconds
elapsed time: 0.00409224 seconds
elapsed time: 0.004327386 seconds
elapsed time: 0.01216568 seconds
elapsed time: 0.004611064 seconds
elapsed time: 0.004621294 seconds
elapsed time: 0.004363505 seconds
elapsed time: 0.004259087 seconds
elapsed time: 0.004787008 seconds
elapsed time: 0.004408419 seconds
elapsed time: 0.004989732 seconds
elapsed time: 0.004513559 seconds
elapsed time: 0.004684414 seconds
elapsed time: 0.004612066 seconds
elapsed time: 0.00439286 seconds
elapsed time: 0.004602738 seconds
elapsed time: 0.00446145 seconds
elapsed time: 0.004396688 seconds
elapsed time: 0.004677019 seconds
elapsed time: 0.0049

elapsed time: 0.011406159 seconds
elapsed time: 0.004602087 seconds
elapsed time: 0.005329505 seconds
elapsed time: 0.004681858 seconds
elapsed time: 0.00444026 seconds
elapsed time: 0.004661449 seconds
elapsed time: 0.004197649 seconds
elapsed time: 0.004012729 seconds
elapsed time: 0.004436723 seconds
elapsed time: 0.004332305 seconds
elapsed time: 0.004119371 seconds
elapsed time: 0.0041501 seconds
elapsed time: 0.003917508 seconds
elapsed time: 0.003915555 seconds
elapsed time: 0.003910605 seconds
elapsed time: 0.004548234 seconds
elapsed time: 0.004543106 seconds
elapsed time: 0.005238774 seconds
elapsed time: 0.004446231 seconds
elapsed time: 0.004671167 seconds
elapsed time: 0.004226755 seconds
elapsed time: 0.01166475 seconds
elapsed time: 0.004254297 seconds
elapsed time: 0.004001257 seconds
elapsed time: 0.00408189 seconds
elapsed time: 0.004222167 seconds
elapsed time: 0.004224501 seconds
elapsed time: 0.003953657 seconds
elapsed time: 0.004809801 seconds
elapsed time: 0.005

elapsed time: 0.004408871 seconds
elapsed time: 0.004545479 seconds
elapsed time: 0.004537614 seconds
elapsed time: 0.00443467 seconds
elapsed time: 0.004681808 seconds
elapsed time: 0.00454059 seconds
elapsed time: 0.004201076 seconds
elapsed time: 0.004256151 seconds
elapsed time: 0.004327035 seconds
elapsed time: 0.004428607 seconds
elapsed time: 0.004423488 seconds
elapsed time: 0.004373273 seconds
elapsed time: 0.004242926 seconds
elapsed time: 0.004632464 seconds
elapsed time: 0.004432265 seconds
elapsed time: 0.004532624 seconds
elapsed time: 0.004683521 seconds
elapsed time: 0.00440363 seconds
elapsed time: 0.004365498 seconds
elapsed time: 0.00428178 seconds
elapsed time: 0.004237957 seconds
elapsed time: 0.004156001 seconds
elapsed time: 0.003951934 seconds
elapsed time: 0.004103892 seconds
elapsed time: 0.004357773 seconds
elapsed time: 0.004821513 seconds
elapsed time: 0.004600353 seconds
elapsed time: 0.004690905 seconds
elapsed time: 0.00481475 seconds
elapsed time: 0.004

LoadError: [91mInterruptException:[39m